## 基于多树GP的特征工程（Feature Construction）

### 评估函数
特征工程是指遗传编程（GP）用于构造机器学习算法所需的特征，而后续的预测操作由机器学习算法来执行。因此，评估函数内部需要内置机器学习算法或特征质量评估函数。
值得一提的是，如果使用机器学习算法进行评估，为了避免过拟合，最好使用交叉验证误差来评估特征工程的效果。

In [1]:
from sklearn.datasets import load_iris

# 假设我们有一个分类问题，输入为X，输出为y
X,y=load_iris(return_X_y=True)

# 求解问题的目标是最大化分类精度
def evalFeatureEngineering(individuals):
    # 创建新特征
    new_features = []
    for ind_num, ind in enumerate(individuals):
        func = gp.compile(expr=ind, pset=pset)
        new_features.append([func(*record) for record in X])
    
    # 转置新特征数组
    new_features = np.transpose(np.array(new_features))
    
    # 使用决策树分类器
    clf = DecisionTreeClassifier()
    clf.fit(new_features, y)
    
    # 使用交叉验证计算误差
    scores = cross_val_score(clf, new_features, y, cv=5)
    
    # 返回平均分类精度
    return scores.mean(),

### 多树GP 个体表示
实现多树GP的方法有很多，最简单的方式是使用列表结构来表示一个个体，列表中的每个元素都是一棵树。
当然，为了后续的可扩展性，更建议创建一个类来表示个体，类中包含一个列表，列表中的每个元素都是一棵树。不过，这里为了简单起见，我们直接使用列表结构。

In [2]:
import operator
import random
import numpy as np
from deap import base, creator, tools, gp, algorithms
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# 创建GP框架的基本组件
pset = gp.PrimitiveSet("MAIN", X.shape[1])
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.random() * 2 - 1)

# 创建一个适应度类和个体类，个体由多棵树组成
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\Users\zhenl\anaconda3\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


### 交叉/变异算子
对于多树GP，很明显我们还需要自己定义交叉和变异算子。这里我们对DEAP中的交叉和变异算子进行简单包装，将其改造成能够处理多棵树的列表结构的算子。

In [3]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)

# 初始化每个个体为一个包含多棵树的列表
def initIndividual(container, func, size):
    return container(gp.PrimitiveTree(func()) for _ in range(size))

# 交叉和变异算子需要能够处理个体的列表结构
def cxOnePointListOfTrees(ind1, ind2):
    for tree1, tree2 in zip(ind1, ind2):
        gp.cxOnePoint(tree1, tree2)
    return ind1, ind2

def mutUniformListOfTrees(individual, expr, pset):
    for tree in individual:
        gp.mutUniform(tree, expr=expr, pset=pset)
    return individual,

toolbox.register("individual", initIndividual, creator.Individual, toolbox.expr, size=3)  # 假设我们创建3个特征
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalFeatureEngineering)  # 假定这个函数已经定义
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", cxOnePointListOfTrees)
toolbox.register("mutate", mutUniformListOfTrees, expr=toolbox.expr, pset=pset)
toolbox.register("compile", gp.compile, pset=pset)

最后，演化流程与传统的GP算法没有区别。直接运行即可。

In [4]:
# 运行遗传编程算法
population = toolbox.population(n=50)
hof = tools.HallOfFame(1)

# 统计和日志
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(population, toolbox, 0.5, 0.2, 40, stats=stats, halloffame=hof, verbose=True)

# 查看最佳个体
best_ind=hof[0]
print('Best individual is:', [str(tree) for tree in best_ind])
print('With fitness:', best_ind.fitness.values)

gen	nevals	avg     	std      	min     	max     
0  	50    	0.904133	0.0863058	0.506667	0.973333
1  	33    	0.934933	0.0267186	0.853333	0.973333
2  	29    	0.943867	0.0262667	0.853333	0.973333
3  	36    	0.942   	0.0282921	0.82    	0.973333
4  	22    	0.949867	0.0366907	0.74    	0.973333
5  	27    	0.946267	0.0519728	0.64    	0.973333
6  	34    	0.942267	0.040587 	0.76    	0.98    
7  	32    	0.951867	0.0285125	0.813333	0.98    
8  	34    	0.9532  	0.0174096	0.913333	0.98    
9  	30    	0.9556  	0.0246075	0.866667	0.98    
10 	27    	0.9532  	0.0251393	0.84    	0.98    
11 	36    	0.9536  	0.0265317	0.813333	0.98    
12 	32    	0.9536  	0.0237378	0.86    	0.98    
13 	32    	0.9504  	0.0303801	0.82    	0.98    
14 	32    	0.946267	0.0346515	0.82    	0.98    
15 	26    	0.953733	0.0282657	0.833333	0.98    
16 	28    	0.950933	0.0474929	0.673333	0.98    
17 	25    	0.953333	0.048037 	0.68    	0.98    
18 	28    	0.9588  	0.0231206	0.886667	0.98    
19 	25    	0.9596  	0.0266803	0.866667	0